# Build CNN with DFG IR

This tutorial illustrates how to construct a Convolutional Neural Network (CNN) by using a framework-independent **Intermediate Representation** (IR) that describes **Data-Flow Graph** (DFG). This IR contains primitives that describe nodes and their connections in the DFG and what operations will these nodes process. Nodes in DFG also carry auxilliary information that directs further optimisation, such as design space exploration.

We will walk through the basic syntax of the IR, present how the CNN built with the IR can be visualised and verified, and show the instructions to train in a TensorFlow back-end. Next tutorials will discuss how CNN described in the DFG IR can be deployed on embedded FPGA devices.

In [1]:
import tensorflow as tf

/home/weizou/learn_keras/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## DFG IR Syntax
**DFG IR** file structure is very simple, it has a name attribute, and node attribute. 
node can be repeated more than one, name can only have one.

For example:
```
name: "test"
node {
  name: "img_input"
  op: "Input"
  device: CPU
  type: T_FLOAT
  input_op_param {
    shape {
      dim: 32
      dim: 1
      dim: 28
      dim: 28
    }
  }
}
node {
  name: "conv1"
  input: "img_input"
  op: "Conv2D"
  device: CPU
  type: T_FLOAT
  conv2d_op_param {
    depth: 32
    kernel_size: 5
    pad: 2
    stride: 1
    activation_fn: "Relu"
    use_bias: true
  }
}
```
In node:
- ``name``: Used to specify the name of the node.
- ``op``: Used to specify node's Operation type.
- ``device``: Specifies the execution device for node, such as ``CPU``, ``GPU``, ``FPGA``.
- ``type``: Used to specify node's data type.
- ``input``: Used to specify the input of the current node.
- ``xxx_op_param``: is a private parameter of node. It has various parameters that only belong to this op. 
  Its name follows the rules of xxx_op_param, xxx is the lowercase of the node's op value, followed by a fixed format. 
  The exception is ``MaxPool2D``, which is ``max_pool2d_op_param``.

### Input Node
If you want to construct an input like tf.placeholder for Tensorflow:


In [2]:
img_input = tf.placeholder(tf.float32, shape=[1, 28, 28, 3], name='img_input')
print("Tensorflow: ", img_input)

Tensorflow:  Tensor("img_input:0", shape=(1, 28, 28, 3), dtype=float32)


In [3]:
# Use DFG IR
from plumber.dfg.nodes import DFGInputNode

input_node = DFGInputNode('img_input', shape=[1, 28, 28, 3])
print("DFG IR: ", input_node.tf_api())
print("------------------------------")
print("DFG IR pbtxt")
print(input_node.as_dfg_node_def())

DFG IR:  Tensor("img_input_1:0", shape=(1, 28, 28, 3), dtype=float32)
------------------------------
DFG IR pbtxt
name: "img_input"
op: "Input"
device: CPU
type: T_FLOAT
input_op_param {
  shape {
    dim: 1
    dim: 28
    dim: 28
    dim: 3
  }
}



### Conv2D Node
- depth: Integer, the dimensionality of the output space.
- kernel_size: Integer, specifying the height and width of the 2D convolution window. We currently only support the same size kernel_size.
- pad: Integer, “valid” is 0, “same” is 1.
- stride: Integer, specifying the strides of the convolution along the height and width. We currently only support the same stride value.
- activation_fn: String, Activate the function name. If it is not needed, fill in the null character.

In [4]:
conv1 = tf.layers.conv2d(img_input, 32, 5, padding='same', name="conv1")
print(conv1)

Tensor("conv1/BiasAdd:0", shape=(1, 28, 28, 32), dtype=float32)


In [6]:
# Use DFG IR
from plumber.dfg.nodes import DFGConv2DNode

conv1_node = DFGConv2DNode('conv1_1', num_filters=32, kernel_size=5, padding=1, stride=1, activation="Relu")
print("DFG IR: ", conv1_node.tf_api(img_input))
print("------------------------------")
print("DFG IR pbtxt")
print(conv1_node.as_dfg_node_def())

DFG IR:  Tensor("conv1_1_1/Relu:0", shape=(1, 28, 28, 32), dtype=float32)
------------------------------
DFG IR pbtxt
name: "conv1_1"
op: "Conv2D"
device: CPU
type: T_FLOAT
conv2d_op_param {
  depth: 32
  kernel_size: 5
  pad: 1
  stride: 1
  activation_fn: "Relu"
  use_maxpool_2x2: false
  use_batch_norm: false
  use_bias: false
  use_relu: true
}



## Visualising DFG IR

In [11]:
with open('mnist.pbtxt', 'r') as f:
    print(f.read())

name: "CNN_MNIST"
node {
  name: "img_input"
  op: "Input"
  device: CPU
  type: T_FLOAT
  input_op_param {
    shape {
      dim: 1
      dim: 28
      dim: 28
      dim: 1
    }
  }
}
node {
  name: "conv1"
  input: "img_input"
  op: "Conv2D"
  device: CPU
  type: T_FLOAT
  conv2d_op_param {
    depth: 32
    kernel_size: 5
    pad: 2
    stride: 1
    activation_fn: "Relu"
    use_bias: true
  }
}
node {
  name: "pool1"
  input: "conv1"
  op: "MaxPool2D"
  device: CPU
  type: T_FLOAT
  max_pool2d_op_param {
    kernel_size: 2
    stride: 2
  }
}
node {
  name: "conv2"
  input: "pool1"
  op: "Conv2D"
  device: CPU
  type: T_FLOAT
  conv2d_op_param {
    depth: 64
    kernel_size: 5
    pad: 2
    stride: 1
    activation_fn: "Relu"
    use_bias: true
  }
}
node {
  name: "pool2"
  input: "conv2"
  op: "MaxPool2D"
  device: CPU
  type: T_FLOAT
  max_pool2d_op_param {
    kernel_size: 2
    stride: 2
  }
}
node {
  name: "flatten"
  input: "pool2"
  op: "Flatten"
  device: CPU
  type: 

## Verifying DFG IR

## Training by TensorFlow

In [10]:
import tensorflow as tf
from plumber.utils import dfg_to_tf

graph = tf.Graph()
graph, output_nodes = dfg_to_tf('mnist.pbtxt', graph)
with tf.Session(graph=graph) as sess:
    graph_def = graph.as_graph_def()
    for node in graph_def.node:
        print(node.name, node.op, list(node.input))

img_input Placeholder []
conv1/weights/Initializer/random_uniform/shape Const []
conv1/weights/Initializer/random_uniform/min Const []
conv1/weights/Initializer/random_uniform/max Const []
conv1/weights/Initializer/random_uniform/RandomUniform RandomUniform ['conv1/weights/Initializer/random_uniform/shape']
conv1/weights/Initializer/random_uniform/sub Sub ['conv1/weights/Initializer/random_uniform/max', 'conv1/weights/Initializer/random_uniform/min']
conv1/weights/Initializer/random_uniform/mul Mul ['conv1/weights/Initializer/random_uniform/RandomUniform', 'conv1/weights/Initializer/random_uniform/sub']
conv1/weights/Initializer/random_uniform Add ['conv1/weights/Initializer/random_uniform/mul', 'conv1/weights/Initializer/random_uniform/min']
conv1/weights VariableV2 []
conv1/weights/Assign Assign ['conv1/weights', 'conv1/weights/Initializer/random_uniform']
conv1/weights/read Identity ['conv1/weights']
conv1/biases/Initializer/zeros Const []
conv1/biases VariableV2 []
conv1/biases/Ass